In [1]:
import cv2
import numpy as np
import os

# Define the interpolation function
def interpolate_frames(video_frames, target_num_frames):
    num_frames = len(video_frames)
    if num_frames >= target_num_frames:
        return video_frames[:target_num_frames]

    # Get evenly spaced indices to interpolate between
    indices = np.linspace(0, num_frames - 1, target_num_frames)
    interpolated_frames = []

    for idx in indices:
        i = int(np.floor(idx))
        alpha = idx - i

        if i + 1 < num_frames:
            frame1 = video_frames[i]
            frame2 = video_frames[i + 1]
            # Blend the two frames
            blended = cv2.addWeighted(frame1, 1 - alpha, frame2, alpha, 0)
            interpolated_frames.append(blended)
        else:
            # For the very last frame (edge case)
            interpolated_frames.append(video_frames[-1])

    return interpolated_frames

# Function to remove duplicate frames
def remove_duplicate_frames(frames):
    unique_frames = []
    prev_frame = None
    for frame in frames:
        # Compare current frame with the previous one (convert to grayscale for easier comparison)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if prev_frame is None or not np.array_equal(gray_frame, prev_frame):
            unique_frames.append(frame)
            prev_frame = gray_frame
    return unique_frames

# Load frames from a folder
def load_frames_from_folder(folder_path):
    frames = []
    for filename in sorted(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, filename)
        if file_path.endswith(".jpg") or file_path.endswith(".png"):
            frame = cv2.imread(file_path)
            if frame is not None:
                frames.append(frame)
    return frames

# Resize frames to a common size
def resize_frames_to_target_size(frames, target_size):
    resized_frames = [cv2.resize(frame, target_size) for frame in frames]
    return resized_frames

# Folders containing preprocessed frames for each video
folders = [
    "preprocessed_data/Unbalance_weight/front",
    "preprocessed_data/Unbalance_weight/angle",
    "preprocessed_data/Bearing_fault/front",
    "preprocessed_data/Bearing_fault/angle",
    "preprocessed_data/Normal_state/front",
    "preprocessed_data/Normal_state/angle"
]

# Load all frames from the folders
all_frames = []
for folder in folders:
    frames = load_frames_from_folder(folder)
    frames = remove_duplicate_frames(frames)  # Remove duplicates before adding to the list
    all_frames.append(frames)

# Find the maximum number of frames among all videos
max_frames = max(len(video_frames) for video_frames in all_frames)
print(f"Max number of frames: {max_frames}")

# Apply frame duplication or interpolation to match the max number of frames
all_resized_frames = [interpolate_frames(video_frames, max_frames) for video_frames in all_frames]

# Resize all frames to a common size (e.g., the size of the first video)
target_height, target_width = all_resized_frames[0][0].shape[:2]  # Use the size of the first video
all_resized_to_same_size_frames = [resize_frames_to_target_size(video_frames, (target_width, target_height)) for video_frames in all_resized_frames]

# Assuming the frames are resized and you want to save them to separate videos
fps = 30  # Choose a common FPS (e.g., 30 FPS)
frame_height, frame_width = target_height, target_width  # Use the common frame size

# List to store the VideoWriter objects for each video
video_writers = []

# Create a VideoWriter object for each folder/video
output_paths = [
    'merged_preprocessed_videos/Unbalance_weight/front.avi',
    'merged_preprocessed_videos/Unbalance_weight/angle.avi',
    'merged_preprocessed_videos/Bearing_fault/front.avi', 
    'merged_preprocessed_videos/Bearing_fault/angle.avi', 
    'merged_preprocessed_videos/Normal_state/front.avi',
    'merged_preprocessed_videos/Normal_state/angle.avi'
]

# Ensure output directories exist
for path in output_paths:
    os.makedirs(os.path.dirname(path), exist_ok=True)

# Create VideoWriter objects
for path in output_paths:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video format (e.g., mp4)
    video_writer = cv2.VideoWriter(path, fourcc, fps, (frame_width, frame_height))
    video_writers.append(video_writer)

# Write frames to each corresponding video
for i, video_frames in enumerate(all_resized_to_same_size_frames):
    for frame in video_frames:
        # Write each frame of the corresponding video to its VideoWriter
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        video_writers[i].write(frame_bgr)

# Release all VideoWriter objects
for writer in video_writers:
    writer.release()

print("Videos saved for each condition.")


Max number of frames: 2819
Videos saved for each condition.
